# Sciebo/NextCloud/Owncloud Folders as a Remote Filesystem

Cloud storage services like Sciebo, NextCloud, and Owncloud offer convenient solutions for storing and accessing large datasets, and they are offered free of charge to all researchers in the NRW region! This section focuses on integrating these services into your data analysis pipeline, demonstrating how to interact with these cloud storage platforms programmatically to manage your neuroscience research data efficiently.

These systems include a "WebDAV" interface for interacting with them as a remote filesystem. In this section, we'll use a `WebdavFileSystem` fsspec class to connect!

If you haven't made a Sciebo account yet, here are some links:

  - https://uni-koeln.sciebo.de/login

  - https://uni-bonn.sciebo.de/login

## Read-Only, Public Datasets

Accessing public datasets is a common requirement in neuroscience research. In Sciebo, you can make one instantly by creating a "Public Link" to a folder. 

  - **URL**: `https://uni-bonn.sciebo.de/public.php/webdav`, 
  - **Username**: The code at the end of the public link.
  - **Passsword**: Whatever password you made for the link, or an empty string if no password.

```python
fs = WebdavFileSystem(url, auth=(username, password))
```

This section provides insights into connecting with read-only, public datasets using WebDAV-based file systems. 



**Exercises**

In [1]:
%pip install webdav4

Note: you may need to restart the kernel to use updated packages.


In [2]:
from webdav4.fsspec import WebdavFileSystem

List all the files in the root directory of this public Sciebo Link: https://uni-bonn.sciebo.de/s/f81JqGZmEHXxMnB

In [3]:
fs = WebdavFileSystem('https://uni-bonn.sciebo.de/public.php/webdav', auth=('f81JqGZmEHXxMnB', 'abc'))

What files are in the "data/final" folder?

In [4]:
fs.glob('data/final/*')

['data/final/steinmetz_all.csv',
 'data/final/steinmetz_all.parquet',
 'data/final/steinmetz_summer2017.csv',
 'data/final/steinmetz_winter2016.csv',
 'data/final/steinmetz_winter2017.csv']

How many files are in the "data/processed" folder?

In [5]:
len(fs.glob('data/processed/*'))

40

## Remote Filesystems that Require Authentication: Basic Secret Handling

It's important not to put the text of the username and password inside your code files; it makes it too easy for others to find (including robots), and makes it more difficult to share code with others.  Instead, we should put secrets it in a seperate file that we won't share with others; something that our code file can use.

Here, we'll try out a standard approach that works in a wide variety of situations: make a `.env` ("dot env") file, which has all the environment variables we want to use.  Because it has a special extension, this file is easy to tell git to ignore, and many tools know how to work with it automatically.


| Code | Description
| :--  | :-- |
| `dotenv.load_dotenv(override=True)` | Put variables in a `.env` file into the operating system's environment variables |
| `os.environ["MyVar"]` | Get an environment variable |

In [17]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


**Exercises**


**Create a Shared Writeable Folder**

Pick one person in your group to create a shared folder for the group to upload data to.  That person should: 
  - Using Sciebo in the web browser, create a new, empty folder in their Sciebo account.
  - Create a new text file in that folder called "README.txt", and put a special message inside the file.
  - **For each** team member to share the code with:
    - Share the folder by creating a "public link", giving the 
        - name of the team member as the link's name, 
        - permission to download/view/upload/edit, 
        - a password, 
        - and set the link to expire in the near-ish future.
    - Using the Zoom Chat, give each person the share link and the password.




**Put the Authentication Data into a .env file** 

Make a file called `.env` and write the username and password into the file as variables like so:

```dotenv
URL=https://uni-bonn.sciebo.de/public.php/webdav  # or whatever the correct address is.
USERNAME=f81JqGZmEHLxMnB                          # the last part of the share link (e.g. from https://uni-bonn.sciebo.de/s/f81JqGZmEHLxMnB)
PASSWORD=mypassword                               # the password
```


**Access your Authentication Data**

Use `load_dotenv()` and `os.environ` to extract the authentication into Python:

In [18]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [12]:
import os

In [18]:
#make the variables available in the environment
import dotenv
from dotenv import load_dotenv
dotenv.load_dotenv()

#get the desired environment variable
import os
os.environ['password']

'blah'

In [23]:
URL=os.environ['URL']
username=os.environ['username_sciebo']
password=os.environ['password']

fs = WebdavFileSystem(URL, auth=(username, password))


**Use Your Authentication Data**

Use the `env` variables to connect to the shared remote folder, and list all the files in that folder.

In [24]:
fs.ls("")

[{'name': 'README.txt',
  'href': '/public.php/webdav/README.txt',
  'size': 18,
  'created': None,
  'modified': datetime.datetime(2024, 2, 22, 15, 8, 11, tzinfo=datetime.timezone.utc),
  'content_language': None,
  'content_type': 'text/plain',
  'etag': '"30deb0d2b228d6ab84aba0a4a1e4a20a"',
  'type': 'file',
  'display_name': None}]

**Analyze the Data!**

Read the text in the README file.  What message is inside the file?

In [35]:
fs.read_text('Mo.txt')

'ICECREAM SOLVES EVERYTHING'

Write text to a file named "\<your_name\>.txt" (e.g. "emma.txt"), and put a hello message inside that file, and check that the file was created!

In [34]:
fs.write_text('README.txt','you are special:P -I know I am ,honey')

37

Create a folder named "images" and upload a picture of your favorite animal to the folder.  Check that it was uploaded properly.

In [37]:
fs.makedirs('images')

FileExistsError: [Errno 17] File exists: 'images'

What did the other people in your group upload?  Take a look, and download their favorite animal images!

In [42]:
image=r"c:/Users/Theodore Tamiolakis/Downloads/image_Theo.jpg"
fs.upload(image,'images/Theo_animal.png')

## Mounting Remote Filesystems as a Network Drive

In Windows, Mac, and Linux, you can mount a remote filesystem so you can browse it in your file explorer.  Let's try it with our shared folder!  This doesn't require any code; when mapping the drive, use the same url, username, and password as we did in the previous exercise.

**Exercise**: Mount one of the WebDAV folders we connected to as a remote drive, using the "Map Network Drive" option in your file explorer / finder.

## (Extra) Other Remote Filesystems

The `fsspec` library supports a wide variety of filesystems, including "SSH" connections (like those used to access linux servers) and "SFTP" connections (like the one provided by the university of Bonn for extra file storage).  No matter where the data is, if you have permission to access it, you can use it!

  - Uni-Bonn Data Storage Services:
    - https://www.hrz.uni-bonn.de/de/services/datenablage-fileservices
    - https://www.hrz.uni-bonn.de/en/all-services/data-storage-fileservices/research-data-infrastructure-fdi
  - Uni-Köln Data Storage Services: 
    - https://fdm.uni-koeln.de/serviceangebot/servicekatalog-1